In [5]:
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt

In [6]:
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import base64
import typing as t
import zlib
import json
import torch

In [11]:
class CFG:
    
    EPOCH = 110
    MODEL_PATH = f"ckpts/fold_0_epoch{EPOCH}.pth"
    
    LR = 3e-5
    EPOCHS = 150
    BATCH_SIZE = 5
    N_TRAIN = 1400 # Take first N_TRAIN images for training, rest for validation


    model_type = "unet++"
    encoder_name = "se_resnext101_32x4d"
    encoder_depth = 4
    decoder_channels = [512, 256, 128, 64]
     
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
import segmentation_models_pytorch as smp
model = smp.UnetPlusPlus(encoder_name=CFG.encoder_name, activation=None, encoder_depth=CFG.encoder_depth, decoder_channels=CFG.decoder_channels,encoder_weights=None)

state_dict = torch.load(CFG.MODEL_PATH)
model.load_state_dict(state_dict, strict=True)

model = model.eval()


# model = model.to(CFG.device)

In [13]:
# Create a sample input tensor
input_shape = (1, 3, 512, 512)  # (batch_size, num_channels, height, width)
input_tensor = torch.randn(input_shape)

# Export the model to ONNX
onnx_path = f"onnx-models/{CFG.model_type}-{CFG.encoder_name}-segformer-{CFG.EPOCH}.onnx"
torch.onnx.export(model, input_tensor, onnx_path, opset_version=11)

print(f"Model converted to ONNX and saved at: {onnx_path}")

/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/segmentation_models_pytorch/base/model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Model converted to ONNX and saved at: onnx-models/unet++-se_resnext101_32x4d-segformer-110.onnx
